In [9]:
# 모듈 불러오기 (필요시 추가)
import os
from pathlib import Path
import torch
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import glob 
# from tqdm.auto import tqdm # tqdm.auto 모듈 추가
from tqdm import tqdm # tqdm.auto 모듈 추가
import gc


In [10]:
# device 변수 정의
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [11]:
dataset_yaml = "AIHub/과수화상병 촬영 이미지/YOLOv11Dataset/dataset.yaml"

In [13]:
%cd "AIHub/과수화상병 촬영 이미지"

[Errno 2] No such file or directory: 'AIHub/과수화상병 촬영 이미지'
/ardor924/AIHub/과수화상병 촬영 이미지


In [16]:
# 파라미터 설정
SAMPLE_RATIO = 0.01
epochs = 100
batch = 8
imgsz = 512
save_period = 10
patience = 50

In [17]:
# 메모리정리 & 가비지컬렉터

gc.collect()

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()  # (선택) CUDA IPC 핸들 정리


In [ ]:

# 1. YOLO 모델 정의 및 훈련
def train_yolo():
    try:
        # YOLOv11n
        model = YOLO('yolo11n.pt') 
    except Exception as e:
        print(f"Error initializing YOLO model: {e}")
        print("Please ensure you have an internet connection to download the model weights.")
        return None

    # 훈련 설정
    train_args = {
        'data': './YOLOv11Dataset/dataset.yaml',
        'epochs': epochs, # 적정에폭 50~100
        'batch': batch, # A100사용시
        'imgsz': imgsz,
        'device': 0 if device.type == 'cuda' else 'cpu',
        'workers': 8,
        'project': 'yolov11_results',
        'name': 'fruit_disease_detection',
        'save_period': save_period,  # 10 epoch마다 체크포인트 저장
        'patience': patience,     # 50 epoch 동안 개선 없으면 조기 종료 (epochs보다 크거나 같게 설정)
        'optimizer': 'auto',  # AdamW, SGD, Adam 등
        'lr0': 0.01,         # 초기 학습률
        'lrf': 0.01,         # 최종 학습률 = lr0 * lrf
        'weight_decay': 0.0005

    }

    project_dir = train_args['project']
    exp_name = train_args['name']
    
    print(f"Starting YOLO training with settings: {train_args}")

    try:
        # 훈련 실행 (Ultralytics의 train 함수는 자체적으로 진행 바를 제공합니다.)
        results_trainer = model.train(**train_args)
        print("Training completed.")

        # Ultralytics 8.x+ 버전에서는 Trainer 객체가 반환되며, save_dir을 바로 사용할 수 있습니다.
        if hasattr(results_trainer, 'save_dir') and results_trainer.save_dir:
            actual_run_dir = Path(results_trainer.save_dir)
            print(f"Training results saved to: {actual_run_dir}")
        else:
            print("Warning: Could not get save_dir directly from results_trainer. Searching for the latest run directory based on project/name pattern.")
            
            project_path = Path(project_dir)
            
            if not project_path.exists():
                print(f"Error: Project directory '{project_path}' was not created during training.")
                return None
            
            # project/name 또는 project/name_N 패턴을 따르는 디렉토리들을 찾습니다.
            potential_run_dirs = sorted([d for d in project_path.iterdir() if d.is_dir() and (d.name == exp_name or d.name.startswith(f"{exp_name}"))],
                                         key=os.path.getmtime, reverse=True)
            
            if potential_run_dirs:
                actual_run_dir = potential_run_dirs[0]
                print(f"Identified potential run directory: {actual_run_dir}")
            else:
                print(f"Error: Could not find any run directory within '{project_path}' matching name '{exp_name}' pattern.")
                return None

        # 실제 실행 디렉토리 내에서 best.pt 경로 구성
        best_model_path = actual_run_dir / 'weights' / 'best.pt'

        # 훈련 후 파일이 존재하는지 최종 확인
        if not best_model_path.exists():
            print(f"Error: Best model file 'best.pt' not found at expected path: {best_model_path}")
            print("Training might not have completed successfully or the path logic is incorrect.")
            # 훈련은 완료되었지만 best.pt가 없는 경우 (예: 짧은 epochs) last.pt를 대신 사용해 볼 수 있습니다.
            last_model_path = actual_run_dir / 'weights' / 'last.pt'
            if last_model_path.exists():
                print(f"Warning: 'best.pt' not found, using 'last.pt' instead: {last_model_path}")
                best_model_path = last_model_path
            else:
                print(f"Error: Neither 'best.pt' nor 'last.pt' found in {actual_run_dir / 'weights'}")
                return None

        print(f"Best model path found at: {best_model_path}")
        return str(best_model_path)

    except Exception as e:
        print(f"An error occurred during training: {e}")
        return None


# 2. 모델 검증 (split_type 인자 추가)
def validate_model(model_path, split_type='val'):
    if not model_path or not Path(model_path).exists():
        print(f"Error: Invalid or non-existent model path provided for validation: {model_path}")
        return None

    try:
        model = YOLO(model_path)
    except Exception as e:
        print(f"Error loading model for validation from {model_path}: {e}")
        return None

    val_args = {
        'data': './YOLOv11Dataset/dataset.yaml',
        'split': split_type,
        'imgsz': imgsz,
        'batch': batch,
        'conf': 0.25,
        'iou': 0.45,
        'device': 0 if device.type == 'cuda' else 'cpu',
        'plots': True
    }

    print(f"Starting model validation on '{split_type}' split...")
    try:
        # validate 함수도 자체 진행 바를 제공합니다.
        metrics = model.val(**val_args)
        print(f"\nValidation Results for '{split_type}' split:")
        print(f"mAP@0.5: {metrics.box.map:.4f}")
        print(f"mAP@0.5:0.95: {metrics.box.map50:.4f}")
        print(f"Precision: {metrics.box.precision.mean():.4f}")
        print(f"Recall: {metrics.box.recall.mean():.4f}")
        return metrics
    except Exception as e:
        print(f"An error occurred during validation on '{split_type}' split: {e}")
        return None


# 3. 예측 및 시각화
def predict_and_visualize(model_path, source_dir='./YOLOv11Dataset/images/test', save_dir='predictions'):
    if not model_path or not Path(model_path).exists():
        print(f"Error: Invalid or non-existent model path provided for prediction: {model_path}")
        return None

    # 디렉토리 생성
    save_path = Path(save_dir)
    save_path.mkdir(exist_ok=True)

    # 모델 로드
    try:
        model = YOLO(model_path)
    except Exception as e:
        print(f"Error loading model for prediction from {model_path}: {e}")
        return None

    # 클래스 이름 매핑
    class_names = {
        0: '정상',
        1: '배검은별무니병',
        2: '배과수화상병',
        3: '사과갈색무늬병',
        4: '사과과수화상병',
        5: '사과부란병',
        6: '사과점무늬낙엽병',
        7: '사과탄저병'
    }

    # 색상 매핑 (BGR 형식)
    colors = {
        0: (0, 255, 0),    # 녹색: 정상 배
        1: (0, 0, 255),    # 빨강: 질병 배
        2: (0, 255, 255),  # 노랑: 정상 사과
        3: (255, 0, 0)     # 파랑: 질병 사과
    }

    # 예측 설정
    conf_threshold = 0.5  # 신뢰도 임계값

    # 테스트 이미지에 대한 예측
    test_images = list(Path(source_dir).glob('*.jpg'))
    if not test_images:
        print(f"Warning: No images found in source directory: {source_dir}")
        return

    print(f"Running predictions on {len(test_images)} test images...")

    # tqdm으로 진행 바 추가
    for img_path in tqdm(test_images, desc="Processing Test Images", unit="image"):
        # 이미지 로드
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"Warning: Could not read image: {img_path}. Skipping.")
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # 예측 수행
        try:
            results = model.predict(
                source=img_rgb,
                conf=conf_threshold,
                imgsz=imgsz,
                device=device,
                verbose=False # 예측 중 출력을 줄입니다.
            )
        except Exception as e:
            print(f"Error during prediction for image {img_path}: {e}")
            continue

        # 결과 시각화 및 출력
        result_img = img.copy()
        
        print(f"\n--- Predictions for {img_path.name} ---")
        detections_found = False # 예측된 객체가 있는지 확인하는 플래그

        if results and len(results) > 0 and hasattr(results[0], 'boxes'):
            for box in results[0].boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().tolist()
                class_id = int(box.cls)
                conf = box.conf.item()

                # 예측 결과 텍스트 출력
                class_name = class_names.get(class_id, f'클래스 {class_id}')
                print(f"  Detected: {class_name}, Confidence: {conf:.4f}, BBox: [{int(x1)}, {int(y1)}, {int(x2)}, {int(y2)}]")
                detections_found = True

                # 박스 그리기
                cv2.rectangle(
                    result_img,
                    (int(x1), int(y1)),
                    (int(x2), int(y2)),
                    colors.get(class_id, (255, 255, 255)),
                    2
                )

                # 라벨 텍스트
                label = f"{class_name}: {conf:.2f}"

                # 라벨 배경
                (label_width, label_height), _ = cv2.getTextSize(
                    label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1
                )
                label_bg_x1 = int(x1)
                label_bg_y1 = int(y1) - label_height - 5
                label_bg_x2 = int(x1) + label_width
                label_bg_y2 = int(y1)
                label_bg_y1 = max(0, label_bg_y1)
                label_bg_y2 = max(label_height + 5, label_bg_y2)

                cv2.rectangle(
                    result_img,
                    (label_bg_x1, label_bg_y1),
                    (label_bg_x2, label_bg_y2),
                    colors.get(class_id, (255, 255, 255)),
                    -1
                )

                # 라벨 텍스트
                cv2.putText(
                    result_img,
                    label,
                    (int(x1), int(y1) - 5 if int(y1) - 5 > label_bg_y1 else label_bg_y2 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (255, 255, 255),
                    1
                )
        
        if not detections_found:
            print("  No objects detected.")

        # 결과 저장
        output_path = save_path / f"pred_{img_path.name}"
        try:
            cv2.imwrite(str(output_path), result_img)
        except Exception as e:
            print(f"Error saving image {output_path}: {e}")

        # 결과 표시 (첫 5개 이미지만)
        if test_images.index(img_path) < 5: # 인덱스 직접 접근은 tqdm과 함께 사용 시 비효율적일 수 있으나, 여기서는 작은 목록이므로 큰 문제 없음.
            plt.figure(figsize=(12, 8))
            plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
            plt.title(f"Predictions: {img_path.name}")
            plt.axis('off')
            plt.show()

    print(f"Predictions saved to: {save_path}")
    return save_path


# 4. 모델 추론 (단일 이미지)
def run_inference(model_path, image_path):
    if not model_path or not Path(model_path).exists():
        print(f"Error: Invalid or non-existent model path provided for inference: {model_path}")
        return []

    if not Path(image_path).exists():
        print(f"Error: Input image file not found for inference: {image_path}")
        return []

    # 모델 로드
    try:
        model = YOLO(model_path) 
    except Exception as e:
        print(f"Error loading model for inference from {model_path}: {e}")
        return []

    # 클래스 이름 매핑 (predict_and_visualize와 동일하게 정의)
    class_names = {
        0: '정상',
        1: '배검은별무니병',
        2: '배과수화상병',
        3: '사과갈색무늬병',
        4: '사과과수화상병',
        5: '사과부란병',
        6: '사과점무늬낙엽병',
        7: '사과탄저병'
    }

    # 이미지 로드
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image for inference: {image_path}")
        return []

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 추론 실행
    print(f"Running inference on image: {image_path}")
    # 단일 이미지 추론에는 tqdm이 불필요합니다.
    try:
        results = model.predict(
            source=img_rgb,
            conf=0.5,
            imgsz=640,
            device=device,
            verbose=False
        )
    except Exception as e:
        print(f"An error occurred during inference for image {image_path}: {e}")
        return []

    # 결과 처리 및 시각화
    detections = []
    result_img = img.copy()

    # 예측 결과 출력 (단일 이미지 추론 결과)
    print(f"\n--- Inference Results for {Path(image_path).name} ---")
    detections_found_inference = False # 단일 이미지 추론에서 객체 탐지 여부 플래그

    if results and len(results) > 0 and hasattr(results[0], 'boxes'):
        for box in results[0].boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().tolist()
            class_id = int(box.cls)
            conf = float(box.conf)

            class_name = class_names.get(class_id, f'클래스 {class_id}') # 클래스 이름 사용
            print(f"  Detected: {class_name}, Confidence: {conf:.4f}, BBox: [{int(x1)}, {int(y1)}, {int(x2)}, {int(y2)}]")
            detections_found_inference = True

            detections.append({
                'class_id': class_id, # 원래 ID 유지
                'class_name': class_name, # 추가된 클래스 이름
                'confidence': conf,
                'bbox': [x1, y1, x2, y2]
            })

            # 시각화: 박스 그리기
            cv2.rectangle(
                result_img,
                (int(x1), int(y1)),
                (int(x2), int(y2)),
                (0, 255, 0), # 녹색 박스 (predict_and_visualize 함수와 색상 다름 주의)
                2
            )
            label = f"{class_name}: {conf:.2f}" # 클래스 이름으로 라벨 표시
            cv2.putText(
                result_img,
                label,
                (int(x1), int(y1) - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2
            )
    
    if not detections_found_inference:
        print("  No objects detected.")

    # 결과 표시
    if detections_found_inference: # 탐지된 객체가 있을 때만 이미지 표시
        plt.figure(figsize=(10, 8))
        plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
        plt.title("Object Detection Result")
        plt.axis('off')
        plt.show()
    else:
        print("No objects detected in the image. Displaying original image.")
        plt.figure(figsize=(10, 8))
        plt.imshow(img_rgb)
        plt.title("No Objects Detected")
        plt.axis('off')
        plt.show()

    return detections


# 메인 실행
if __name__ == "__main__":
    # cv2 및 matplotlib import 추가 (predict_and_visualize, run_inference 함수에서 사용)
    import cv2
    import matplotlib.pyplot as plt

    # device 변수가 main 함수 범위에서 정의되어 있는지 확인 (이미 맨 위에 정의됨.)

    # 1. 모델 훈련
    print("Starting model training...")
    trained_model_path = train_yolo() # train_yolo는 Ultralytics 내부 진행 바 사용

    if trained_model_path:
        print(f"\nSuccessfully trained model saved at: {trained_model_path}")

        # 2. 검증 (Validation set)
        print("\nValidating on validation set...")
        val_metrics = validate_model(trained_model_path, split_type='val') # validate_model은 Ultralytics 내부 진행 바 사용
        if val_metrics:
            print("Validation complete.")
        else:
            print("Validation failed or returned no metrics.")

        # 3. 테스트 (Test set)
        print("\nTesting on test set...")
        test_metrics = validate_model(trained_model_path, split_type='test') # validate_model은 Ultralytics 내부 진행 바 사용
        if test_metrics:
            print("Test complete.")
        else:
            print("Testing failed or returned no metrics.")

        # 4. 예측 시각화
        print("\nRunning predictions on test images...")
        # predict_and_visualize 함수 내부에서 tqdm 진행 바가 출력됩니다.
        predict_save_path = predict_and_visualize(trained_model_path)
        if predict_save_path:
            print(f"Prediction images saved to: {predict_save_path}")
        else:
            print("Prediction and visualization failed.")

        # 5. 단일 이미지 추론 예시
        test_image_files = list(Path('./YOLOv11Dataset/images/test').glob('*.jpg'))
        if test_image_files:
            sample_image = test_image_files[0]
            print(f"\nRunning inference on sample image: {sample_image}")
            detections_single_image = run_inference(trained_model_path, str(sample_image))
            
        else:
            print("\nNo test images found for single image inference example.")

    else:
        print("\nModel training failed. Cannot proceed with validation, testing, prediction, or inference.")

Starting model training...
Starting YOLO training with settings: {'data': './YOLOv11Dataset/dataset.yaml', 'epochs': 100, 'batch': 8, 'imgsz': 512, 'device': 0, 'workers': 8, 'project': 'yolov11_results', 'name': 'fruit_disease_detection', 'save_period': 10, 'patience': 50, 'optimizer': 'auto', 'lr0': 0.01, 'lrf': 0.01, 'weight_decay': 0.0005}
New https://pypi.org/project/ultralytics/8.3.158 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.156 🚀 Python-3.8.10 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100 80GB PCIe MIG 3g.40gb, 40192MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./YOLOv11Dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, f

train: Scanning /ardor924/AIHub/과수화상병 촬영 이미지/YOLOv11Dataset/labels/train... 59240 images, 0 backgrounds, 164 corrupt: 100%|██████████| 59240/59240 [23:45<00:00, 41.56it/s]

train: /ardor924/AIHub/과수화상병 촬영 이미지/YOLOv11Dataset/images/train/V006_80_0_00_01_05_25_0_c06_20201020_0483_S01_1.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.041]
train: /ardor924/AIHub/과수화상병 촬영 이미지/YOLOv11Dataset/images/train/V006_80_0_00_01_05_25_0_c06_20201029_0317_S01_1.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0003]
train: /ardor924/AIHub/과수화상병 촬영 이미지/YOLOv11Dataset/images/train/V006_80_0_00_01_05_25_0_c13_20201026_0209_S01_1.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.002]
train: /ardor924/AIHub/과수화상병 촬영 이미지/YOLOv11Dataset/images/train/V006_80_0_00_01_05_25_0_c13_20201026_0223_S01_1.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0002]
train: /ardor924/AIHub/과수화상병 촬영 이미지/YOLOv11Dataset/images/train/V006_80_0_00_01_05_25_0_c16_20201103_0013_S01_1.JPG: ignoring corrupt image/label: non-normalized or out of bounds c

train: New cache created: /ardor924/AIHub/과수화상병 촬영 이미지/YOLOv11Dataset/labels/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 17.2±7.3 MB/s, size: 1670.9 KB)


val: Scanning /ardor924/AIHub/과수화상병 촬영 이미지/YOLOv11Dataset/labels/val.cache... 3709 images, 0 backgrounds, 12 corrupt: 100%|██████████| 3709/3709 [00:00<?, ?it/s]

...

                   all       3697       3697      0.885      0.735      0.813      0.664

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     28/100      1.42G     0.7783     0.7734      1.138          9        512: 100%|██████████| 7385/7385 [34:26<00:00,  3.57it/s]  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 232/232 [02:04<00:00,  1.86it/s]

                   all       3697       3697      0.872      0.741      0.813      0.665



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     29/100      1.42G     0.7839     0.7789      1.141         22        512:  30%|██▉       | 2200/7385 [08:26<13:50,  6.24it/s]  